# Process FFT Datasets Into Images

Here, we go through the classed functions for formatting the fft datasets into their image representations. We can:

- map data using the xyz coordinates using a PCA method
- map data using the inverse gain matrix transposed to map onto the parcellated 84 regions in MRI xyz space

These methods can be compared to determine the accuracy of encoding different types of structural information into the models.

In [2]:
import numpy as np
import os
import sys

sys.path.append('../')
from dnn.processing.format import formatfft

%load_ext autoreload
%autoreload 2

/Users/adam2392/Documents/dnn-unsupervised/.venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
def main(fftdatadir, rawdatadir,  metadatadir, outputdatadir):
    formatter = formatfft.FormatFFT(fftdatadir, rawdatadir, metadatadir, outputdatadir)

    formatter.getdatafiles()
    formatter.formatdata()
    print('finished!')

In [30]:
# patid = 'id001_ac'

# the main data directories that there is raw, meta, and output data
metadatadir = '/Volumes/ADAM LI/pydata/metadata/'
rawdatadir = '/Volumes/ADAM LI/pydata/convertedtng/'
fftdatadir = '/Volumes/ADAM LI/pydata/output_fft/tngcenter/win500_step250/'
# fftdatadir = '/Volumes/ADAM LI/pydata/output_fft/tvbsim/full/'
outputdatadir = '/Volumes/ADAM LI/pydata/output_fft/asimages/regions/'

# confile = os.path.join(metadatadir, patid, 'connectivity.zip')
# gainmatfile = os.path.join(metadatadir, patid, 'gain_inv-square.txt')
# run conversion
main(fftdatadir, rawdatadir, metadatadir, outputdatadir)

0
Patient is:  id001_ac
file id is:  id001_ac_absence
['power', 'freqs', 'timepoints', 'phase', 'chanlabels']


value error in birth date probably, so just setting to nan

(70, 129, 1799)
Converttoregs summary of data:
power shape:  (70, 5, 1799)
regions xyz shape:  (84, 3)
gainmat shape:  (84, 70)
image tensor shape:  (84, 5, 1799)
Y labels shape:  (1799, 1)
(84, 5, 1799)
saved at  /Volumes/ADAM LI/pydata/output_fft/asimages/regions/id001_ac_absence_fftmodel
1
Patient is:  id001_ac
file id is:  id001_ac_comp
['power', 'freqs', 'timepoints', 'phase', 'chanlabels']


value error in birth date probably, so just setting to nan

(70, 129, 2999)
Converttoregs summary of data:
power shape:  (70, 5, 2999)
regions xyz shape:  (84, 3)
gainmat shape:  (84, 70)
image tensor shape:  (84, 5, 2999)
Y labels shape:  (2999, 1)
(84, 5, 2999)
saved at  /Volumes/ADAM LI/pydata/output_fft/asimages/regions/id001_ac_comp_fftmodel
2
Patient is:  id001_ac
file id is:  id001_ac_sz2


...

value error in birth date probably, so just setting to nan

Patient is:  id012_pc
file id is:  id012_pc_sz1
['power', 'freqs', 'timepoints', 'phase', 'chanlabels']
(132, 257, 1195)
Converttoregs summary of data:
power shape:  (132, 5, 1195)
regions xyz shape:  (84, 3)
gainmat shape:  (84, 132)
image tensor shape:  (84, 5, 1195)
Y labels shape:  (1195, 1)
(84, 5, 1195)
saved at  /Volumes/ADAM LI/pydata/output_fft/asimages/regions/id012_pc_sz1_fftmodel
17


value error in birth date probably, so just setting to nan

Patient is:  id013_pg
file id is:  id013_pg_sz1
['power', 'freqs', 'timepoints', 'phase', 'chanlabels']
(51, 129, 1195)
Converttoregs summary of data:
power shape:  (51, 5, 1195)
regions xyz shape:  (84, 3)
gainmat shape:  (84, 51)
image tensor shape:  (84, 5, 1195)
Y labels shape:  (1195, 1)
(84, 5, 1195)
saved at  /Volumes/ADAM LI/pydata/output_fft/asimages/regions/id013_pg_sz1_fftmodel
18


FileNotFoundError: File b'/Volumes/ADAM LI/pydata/convertedtng/id014_rb_sz1/id014_rb_sz1_chans.csv' does not exist

In [ ]:
    # load in the onset/offset times
    dataloader = LoadPat(patient=fileid, datadir=rawdatadir)
    onset_times = dataloader.onset_time
    offset_times = dataloader.offset_time
#     print(dataloader.onset_time)
#     print(dataloader.offset_time)
    print("Patient is: ", patient)
    print("file id is: ", fileid)
#     print(dirname)
#     print("Filename loaded is: ", filename)
    print("\n\n")
    
    # get indices of channels that we have seeg_xyz for
    chanindices = [i for i,x in enumerate(rawchans) if any(thing == x for thing in seeg_labels)]
    chanlabels = rawchans[chanindices]
    power = np.abs(power[chanindices,:,:])
    
    # get overlapping indices on seeg with xyz
    xyzinds = [i for i,x in enumerate(seeg_labels) if any(thing==x for thing in chanlabels)]
    seeg_xyz = seeg_xyz[xyzinds,:]
 # instantiate metadata hash table
    metadata = dict()
    metadata['chanlabels'] = chanlabels
    metadata['seeg_xyz'] = seeg_xyz
    metadata['ylabels'] = ylabels
    metadata['samplerate'] = samplerate
    metadata['timepoints'] = timepoints
